# 🛡️ Rail B Training: 7-Label Policy Classification (Balanced)

**Dataset**: Balanced Full (210k samples) | **Model**: LFM2-1.2B + LoRA | **Labels**: 7 (no PromptAttack)
**Strategy**: Rarest-label upsampling (Target 15k/class) + 50/50 Safe/Violation split

In [ ]:
import os, sys, torch, torch.nn as nn, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
from sklearn.metrics import f1_score, hamming_loss, classification_report

# === CONFIGURATION ===
MODEL_ID = "LiquidAI/LFM2-1.2B"
DATA_PREFIX = "../data/processed/rail_b_full"
OUTPUT_DIR = "../models/rail_b_v1"

MAX_LENGTH = 512
BATCH_SIZE = 8
GRADIENT_ACCUMULATION = 8  # Effective batch = 64
EPOCHS = 2  # Reduced to 2 epochs since dataset is larger (210k) and balanced
LR = 2e-4

# 7 LABELS ONLY - PromptAttack excluded (handled by Rail A)
CATS = ["Hate", "Harassment", "Sexual", "ChildSafety", "Violence", "Illegal", "Privacy"]
NUM_LABELS = len(CATS)  # 7

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/final", exist_ok=True)

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Device: {device}")
print(f"Labels ({NUM_LABELS}): {CATS}")

## 📊 Data Loading & EDA

In [ ]:
# Load data
train_df = pd.read_parquet(f"{DATA_PREFIX}_train.parquet")
val_df = pd.read_parquet(f"{DATA_PREFIX}_val.parquet")
test_df = pd.read_parquet(f"{DATA_PREFIX}_test.parquet")

print(f"Train: {len(train_df):,} | Val: {len(val_df):,} | Test: {len(test_df):,}")

# Verify label vector length
sample_vec = train_df.iloc[0]['label_vector']
print(f"\n✅ Label vector length: {len(sample_vec)} (expected: {NUM_LABELS})")
assert len(sample_vec) == NUM_LABELS, f"ERROR: Label vector has {len(sample_vec)} elements, expected {NUM_LABELS}"

In [ ]:
# Class Distribution Analysis
label_matrix = np.vstack(train_df['label_vector'].values)

print("=" * 50)
print("CLASS DISTRIBUTION (Training Set)")
print("=" * 50)

class_counts = label_matrix.sum(axis=0)
total_samples = len(train_df)

for i, cat in enumerate(CATS):
    count = int(class_counts[i])
    pct = (count / total_samples) * 100
    print(f"{cat:15s}: {count:>7,} ({pct:>5.1f}%)")

# Safe samples (all zeros)
safe_count = (label_matrix.sum(axis=1) == 0).sum()
print(f"{'SAFE':15s}: {safe_count:>7,} ({safe_count/total_samples*100:>5.1f}%)")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart
colors = plt.cm.Set2(np.linspace(0, 1, NUM_LABELS))
axes[0].bar(CATS, class_counts, color=colors)
axes[0].set_title('Per-Category Sample Count')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Labels per sample
labels_per_sample = label_matrix.sum(axis=1)
axes[1].hist(labels_per_sample, bins=range(0, 8), color='steelblue', edgecolor='black')
axes[1].set_title('Labels per Sample')
axes[1].set_xlabel('Number of Labels')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

print(f"\nAvg labels per sample: {labels_per_sample.mean():.2f}")
print(f"Multi-label samples (2+): {(labels_per_sample >= 2).sum():,}")

## 🏗️ Model Definition

In [ ]:
class SentinelLFMMultiLabel(nn.Module):
    def __init__(self, model_id, num_labels):
        super().__init__()
        self.num_labels = num_labels
        self.base_model = AutoModel.from_pretrained(model_id, trust_remote_code=True)
        self.config = self.base_model.config
        hidden_size = self.config.hidden_size
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, num_labels)
        )
        self.loss_fct = nn.BCEWithLogitsLoss()
    
    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        hidden_states = outputs[0] if isinstance(outputs, tuple) else outputs.last_hidden_state
        if attention_mask is not None:
            last_idx = attention_mask.sum(1) - 1
            pooled = hidden_states[torch.arange(input_ids.shape[0], device=input_ids.device), last_idx]
        else:
            pooled = hidden_states[:, -1, :]
        logits = self.classifier(pooled)
        loss = self.loss_fct(logits, labels.float()) if labels is not None else None
        from transformers.modeling_outputs import SequenceClassifierOutput
        return SequenceClassifierOutput(loss=loss, logits=logits)

## 📦 Tokenization

In [ ]:
def prepare_dataset(df):
    df = df.copy()
    df['labels'] = df['label_vector'].apply(lambda x: x.tolist())
    return Dataset.from_pandas(df[['text', 'labels']])

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, max_length=MAX_LENGTH)

print("Tokenizing...")
tokenized_train = prepare_dataset(train_df).map(preprocess, batched=True, remove_columns=["text"], num_proc=4)
tokenized_val = prepare_dataset(val_df).map(preprocess, batched=True, remove_columns=["text"], num_proc=4)
print("✅ Done")

## 🚀 Model Initialization

In [ ]:
model = SentinelLFMMultiLabel(MODEL_ID, num_labels=NUM_LABELS)
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["out_proj", "v_proj", "q_proj", "k_proj"],
    lora_dropout=0.1,
    bias="none"
)
model.base_model = get_peft_model(model.base_model, peft_config)
model.base_model.print_trainable_parameters()
model.to(device)
print(f"✅ Model ready with {NUM_LABELS} output labels")

## 🎯 Training

In [ ]:
def compute_metrics(eval_pred):
    """Compute comprehensive multi-label metrics."""
    logits, labels = eval_pred
    preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    
    # Overall metrics
    f1_micro = f1_score(labels, preds, average='micro', zero_division=0)
    f1_macro = f1_score(labels, preds, average='macro', zero_division=0)
    hamming = hamming_loss(labels, preds)
    
    # Per-category F1
    f1_per_cat = f1_score(labels, preds, average=None, zero_division=0)
    
    metrics = {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'hamming': hamming,
    }
    
    # Add per-category F1 (7 categories)
    for i, cat in enumerate(CATS):
        metrics[f'f1_{cat.lower()}'] = f1_per_cat[i]
    
    return metrics

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_steps=100,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    dataloader_num_workers=0,
    push_to_hub=False,
    warmup_steps=1000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

print("🚀 Starting training...")
trainer.train()

## 📈 Results & Per-Category Analysis

In [ ]:
log_history = trainer.state.log_history
eval_metrics = [x for x in log_history if 'eval_f1_micro' in x]

if eval_metrics:
    final = eval_metrics[-1]
    
    print("=" * 50)
    print("FINAL EVALUATION METRICS")
    print("=" * 50)
    print(f"F1 Micro: {final['eval_f1_micro']:.4f}")
    print(f"F1 Macro: {final['eval_f1_macro']:.4f}")
    print(f"Hamming Loss: {final['eval_hamming']:.4f}")
    
    print("\n" + "=" * 50)
    print("PER-CATEGORY F1 SCORES")
    print("=" * 50)
    
    cat_f1 = []
    for cat in CATS:
        f1 = final.get(f'eval_f1_{cat.lower()}', 0)
        cat_f1.append(f1)
        print(f"{cat:15s}: {f1:.4f}")
    
    # Visualization
    fig, ax = plt.subplots(figsize=(10, 5))
    colors = ['green' if f1 > 0.7 else 'orange' if f1 > 0.5 else 'red' for f1 in cat_f1]
    ax.barh(CATS, cat_f1, color=colors)
    ax.set_xlabel('F1 Score')
    ax.set_title('Per-Category F1 Scores')
    ax.set_xlim(0, 1)
    ax.axvline(x=0.7, color='green', linestyle='--', alpha=0.5, label='Good (0.7)')
    ax.axvline(x=0.5, color='orange', linestyle='--', alpha=0.5, label='Fair (0.5)')
    ax.legend()
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/per_category_f1.png", dpi=150)
    plt.show()

## 💾 Save Model

In [ ]:
final_dir = f"{OUTPUT_DIR}/final"
torch.save(model.classifier.state_dict(), f"{final_dir}/classifier.pt")
model.base_model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)
torch.save(model.state_dict(), f"{final_dir}/full_model.pt")

# Save category names
import json
with open(f"{final_dir}/categories.json", 'w') as f:
    json.dump({'categories': CATS, 'num_labels': NUM_LABELS}, f, indent=2)

print(f"✅ Model saved to {final_dir}")
print(f"   - classifier.pt")
print(f"   - full_model.pt")
print(f"   - categories.json")

## 🧪 Quick Test

In [ ]:
model.eval()

test_cases = [
    "I hate all those people from that country.",
    "Can you help me write a Python script?",
    "Send me explicit nude photos.",
    "How do I make a homemade explosive?",
    "You're such an idiot, kill yourself.",
    "The weather is nice today.",
]

print("=" * 60)
print("QUICK TEST (Threshold = 0.5)")
print("=" * 60)

for text in test_cases:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).to(device)
    with torch.no_grad():
        probs = torch.sigmoid(model(**inputs).logits)[0].cpu()
    
    detected = [(CATS[i], f"{p:.2f}") for i, p in enumerate(probs) if p > 0.5]
    
    print(f"\n\"{text[:50]}...\"" if len(text) > 50 else f"\n\"{text}\"")
    if detected:
        print(f"  🚨 {', '.join([f'{c} ({p})' for c, p in detected])}")
    else:
        print(f"  ✅ SAFE")